In [ ]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import time
import json


In [ ]:
url = "https://www.fotmob.com/api/leagueseasondeepstats?id=9296&season=24600&type=players&stat=rating&slug=usl-league-one"
payload = {}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36",
    "Referer": "https://www.fotmob.com/leagues/9296/stats/season/24600/players/rating/usl-league-one",
    "x-mas": "eyJib2R5Ijp7InVybCI6Ii9hcGkvbGVhZ3Vlc2Vhc29uZGVlcHN0YXRzP2lkPTkyOTYmc2Vhc29uPTI0NjAwJnR5cGU9cGxheWVycyZzdGF0PXJhdGluZyZzbHVnPXVzbC1sZWFndWUtb25lIiwiY29kZSI6MTc0Nzc1MDYwNDkzMywiZm9vIjoicHJvZHVjdGlvbjo4OTMzMDRhNDBhYmUyZTM4YzJmOTE1NTk4YzAzMDZmZjMzZjRmYjE1LXVuZGVmaW5lZCJ9LCJzaWduYXR1cmUiOiIzOTZCNkFBODUwMjBBREFBNUVDMkVFM0NCODIzMDc5RCJ9",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-ch-ua-mobile": "?0"
}

response = requests.request("GET", url, headers=headers, data=payload)
response.raise_for_status()
print("\nStatus code:", response.status_code)



Status code: 200


In [ ]:
rating=response.json()['statsData']

In [ ]:
ratings = pd.DataFrame(rating)
ratings.head()

,id,teamId,name,substatValue,statValue,rank,type
0,1542126,614315,Luis Alvarez,"{'value': 2, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 7.89, 'format': 'f...",1,players
1,1346899,1584038,Pierre Reedy,"{'value': 1, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 7.72, 'format': 'f...",2,players
2,734061,1358088,Mark Doyle,"{'value': 1, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 7.56, 'format': 'f...",3,players
3,1520643,1584038,David Garcia,"{'value': 1, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 7.52, 'format': 'f...",4,players
4,1240291,1004918,Joshua Ramos,"{'value': 1, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 7.48, 'format': 'f...",5,players


In [ ]:
ratings_test = []
for rat in rating:
  ratings_test.append({'id': rat['id'], 'team_id': rat['teamId'], 'name': rat['name'], 'rating': rat['statValue']['value']})

df=pd.DataFrame(ratings_test)
df


,id,team_id,name,rating
0,1542126,614315,Luis Alvarez,7.89
1,1346899,1584038,Pierre Reedy,7.72
2,734061,1358088,Mark Doyle,7.56
3,1520643,1584038,David Garcia,7.52
4,1240291,1004918,Joshua Ramos,7.48
...,...,...,...,...
146,1742276,1701883,Titus Washington,6.24
147,729742,1584038,Neco Brett,6.16
148,1252140,1701884,Davey Mason,6.11
149,1730812,1701884,Brandon McManus,5.97


In [ ]:
df.to_csv('ratsApril.csv', index=False)

In [ ]:
# MAKE PLAYER LINKS #

links = []
for index, row in ratings.iterrows():
  p_id = row['id']
  player_name = row['name'].lower().replace(' ', '-')

  link = f'https://www.fotmob.com/players/{p_id}/{player_name}'
  links.append(link)

In [ ]:
len(links)

151

In [ ]:
df['links'] = links

In [ ]:
df.head()

,id,team_id,name,rating,links
0,1542126,614315,Luis Alvarez,7.89,https://www.fotmob.com/players/1542126/luis-al...
1,1346899,1584038,Pierre Reedy,7.72,https://www.fotmob.com/players/1346899/pierre-...
2,734061,1358088,Mark Doyle,7.56,https://www.fotmob.com/players/734061/mark-doyle
3,1520643,1584038,David Garcia,7.52,https://www.fotmob.com/players/1520643/david-g...
4,1240291,1004918,Joshua Ramos,7.48,https://www.fotmob.com/players/1240291/joshua-...


In [ ]:
def fetch_player_details(df):
    player_details = []

    for idx, row in df.iterrows():
        player_id = row['id']
        player_name = row['name']

        url = f"https://www.fotmob.com/api/playerData?id={player_id}"
        headers = {
            "sec-ch-ua-platform": "\"Windows\"",
            "Referer": row['links'],
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
            "sec-ch-ua": "\"Google Chrome\";v=\"135\", \"Not-A.Brand\";v=\"8\", \"Chromium\";v=\"135\"",
            "sec-ch-ua-mobile": "?0",
            "x-mas": "eyJib2R5Ijp7InVybCI6Ii9hcGkvcGxheWVyRGF0YT9pZD0xNTQyMTI2IiwiY29kZSI6MTc0Nzc1MDg5Mjg2MywiZm9vIjoicHJvZHVjdGlvbjo4OTMzMDRhNDBhYmUyZTM4YzJmOTE1NTk4YzAzMDZmZjMzZjRmYjE1LXVuZGVmaW5lZCJ9LCJzaWduYXR1cmUiOiI1ODUyMzk5Q0JBNDA0NTNBRjk4QjA3RTA1MkRCMDdEMSJ9"
        }

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()

            position = None

            if 'positionDescription' in data and 'primaryPosition' in data['positionDescription']:
                position = data['positionDescription']['primaryPosition'].get('label', '')

            player_details.append({
                'id': player_id,
                'position': position
            })

            time.sleep(0.5)

        except Exception as e:
            print(f"Error processing {player_name}")

    details_df = pd.DataFrame(player_details)

    result_df = df.merge(details_df, on='id', how='left')

    return result_df

In [ ]:
testdrive = df.head(3)
testdrive

,id,team_id,name,rating,links
0,1542126,614315,Luis Alvarez,7.89,https://www.fotmob.com/players/1542126/luis-al...
1,1346899,1584038,Pierre Reedy,7.72,https://www.fotmob.com/players/1346899/pierre-...
2,734061,1358088,Mark Doyle,7.56,https://www.fotmob.com/players/734061/mark-doyle


In [ ]:
# Get position data #
pos = fetch_player_details(testdrive)

In [ ]:
pos

,id,team_id,name,rating,links,position
0,1542126,614315,Luis Alvarez,7.89,https://www.fotmob.com/players/1542126/luis-al...,Attacking Midfielder
1,1346899,1584038,Pierre Reedy,7.72,https://www.fotmob.com/players/1346899/pierre-...,Left Winger
2,734061,1358088,Mark Doyle,7.56,https://www.fotmob.com/players/734061/mark-doyle,Left Winger


In [ ]:
pos = fetch_player_details(df)

In [ ]:
pos.to_csv('usl1_pos.csv', index=False)